In [ ]:
%matplotlib nbagg
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from simulator import Simulator

In [ ]:
modes = ["coop", "unaware", "aware"]
num_robots = [2, 4, 6, 8, 10]
no_noise_output = [ [ [] for m in modes] for n in num_robots]

# No noise, full visibility
start = time.time()
for i in range(len(num_robots)):
    start = time.time()
    for mode in range(len(modes)):
        sim = Simulator(num_robots[i])
        steps, collisions = sim.run(mode=modes[mode])
        no_noise_output[i][mode].append((steps, collisions))
    end = time.time()
    print(end - start)
print(no_noise_output)
# [{'coop': [(666, 0)], 
# 'unaware': [(673, 0)], 
# 'aware': [(673, 0)]}, 
# {'coop': [(667, 0)], 
# 'unaware': [(669, 0)], 
# 'aware': [(669, 0)]}, 
# {'coop': [(671, 0)], 
# 'unaware': [(678, 52)], 
# 'aware': [(678, 52)]}, 
# {'coop': [(691, 0)], 
# 'unaware': [(707, 232)], 
# 'aware': [(707, 232)]}]

In [ ]:
noise_values = [0.0, 0.01, 0.03, 0.05]
num_robots_for_noise = [2, 4, 6, 8, 10]
noise_output = [ [ [ [] for m in modes] for n in num_robots_for_noise] for n in noise_values ]
for repetition in range(20):
    start = time.time()
    for n in range(len(noise_values)):
        for i in range(len(num_robots_for_noise)):
            for mode in range(len(modes)):
                sim = Simulator(num_robots_for_noise[i])
                steps, collisions = sim.run(mode=modes[mode], current_noise=noise_values[n])
                noise_output[n][i][mode].append((steps, collisions))
    end = time.time()
    print("Step %d done, took %f" % (repetition, (end-start)) )
    

In [ ]:
for n in range(len(noise_values)):
    for i in range(len(num_robots_for_noise)):
        for m in range(len(modes)):
            val = noise_output[n][i][m]
            minima = np.amin(val, axis=0)[0]
            med = np.median(val, axis=0)[0]
            maxima = np.amax(val, axis=0)[0]
            print("Noise: " + str(noise_values[n]),num_robots_for_noise[i],modes[m],noise_output[n][i][m])
            print(minima, med, maxima)

In [ ]:
noise_values = [0.0, 0.01, 0.03, 0.05]
# Plot of min, median and max time taken to get to the goal, 
# divided up by noise in the system and mode of operation of the robots
fig, axes = plt.subplots(3,4)
outsideY = ["Cooperative", "Unaware", "Mitigating"]
outsideX = ["Noise: %.2f" % n for n in noise_values ]
pad = 5 # in points
for ax, col in zip(axes[0], outsideX):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')

for ax, row in zip(axes[:,0], outsideY):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center', rotation='vertical')
plt.setp(axes.flat, xlabel='Number of Robots', ylabel='Time, seconds')
for n in range(len(noise_values)):
    for m in range(len(modes)):
        mine = [[] for i in num_robots_for_noise]
        med = [[] for i in num_robots_for_noise]
        maxe = [[] for i in num_robots_for_noise]
        for i in range(len(num_robots_for_noise)):
            val = noise_output[n][i][m]
            mine[i].append(np.amin(val, axis=0)[0] / 10.0)
            med[i].append(np.median(val, axis=0)[0] / 10.0)
            maxe[i].append(np.amax(val, axis=0)[0] / 10.0)
        axes[m][n].set_xticks(num_robots_for_noise)
        axes[m][n].plot(num_robots_for_noise, mine, 'rv', num_robots_for_noise, med, 'bo', num_robots_for_noise, maxe, 'g^')
        
fig.tight_layout()
fig.subplots_adjust(left=0.15, top=0.95)
fig.savefig('times_to_num_robots_over_modes_and_noise.eps', bbox_inches='tight',  format='eps', dpi=300)
plt.show()

In [ ]:
# Collisions against number of robots, with malicious, aware-oracle and aware-history subplots
noise_values_collision = [0.0, 0.01, 0.03, 0.05]
num_robots_for_collision = [2, 4, 6, 8, 10]
collision_modes = ["unaware", "aware-oracle", "aware-history"]
collision_output = [ [ [ [] for m in collision_modes] for n in num_robots_for_collision ] for n in noise_values_collision ]
for repetition in range(20):
    start = time.time()
    for n in range(len(noise_values_collision)):
        if n == 0 and repetition != 0:
            continue
        for i in range(len(num_robots_for_collision)):
            for mode in range(len(collision_modes)):
                sim = Simulator(num_robots_for_collision[i])
                if collision_modes[mode] == "unaware":
                    steps, collisions = sim.run(mode="coop", current_noise=noise_values_collision[n])
                elif collision_modes[mode] == "aware-oracle":
                    steps, collisions = sim.run(mode="aware", current_noise=noise_values_collision[n], identifier="oracle")
                else:
                    steps, collisions = sim.run(mode="aware", current_noise=noise_values_collision[n], identifier="history")
                collision_output[n][i][mode].append((steps,collisions))
    end = time.time()
    print("Step %d done, took %f" % (repetition, (end-start)) )

In [ ]:
# Plot of min, median and max time taken to get to the goal, 
# divided up by noise in the system and mode of operation of the robots
fig, axes = plt.subplots(3,4)
outsideY = ["Unaware", "Aware-Oracle", "Aware-History"]
outsideX = ["Noise: %.2f" % n for n in noise_values_collision ]
pad = 5 # in points
for ax, col in zip(axes[0], outsideX):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')

for ax, row in zip(axes[:,0], outsideY):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center', rotation='vertical')
plt.setp(axes.flat, xlabel='Number of Robots', ylabel='Collisions')
# Data
for n in range(len(noise_values_collision)):
    for m in range(len(collision_modes)):
        mine = [[] for i in num_robots_for_collision]
        med = [[] for i in num_robots_for_collision]
        maxe = [[] for i in num_robots_for_collision]
        for i in range(len(num_robots_for_collision)):
            val = collision_output[n][i][m]
            mine[i].append(np.amin(val, axis=0)[1])
            med[i].append(np.median(val, axis=0)[1])
            maxe[i].append(np.amax(val, axis=0)[1])
        axes[m][n].set_xticks(num_robots_for_collision)
        axes[m][n].plot(num_robots_for_collision, mine, 'rv', num_robots_for_collision, med, 'bo', num_robots_for_collision, maxe, 'g^')
fig.tight_layout()
# tight_layout doesn't take these labels into account. We'll need 
# to make some room. These numbers are are manually tweaked. 
# You could automatically calculate them, but it's a pain.
fig.subplots_adjust(left=0.2, top=0.95)
fig.savefig('collisions_to_num_robots_over_modes_and_noise.eps', bbox_inches='tight',  format='eps', dpi=300)
plt.show()

In [ ]:
# Collisions against visibility range, with 4 robots, with malicious, aware-oracle and aware-history subplots
noise_values_visibility = [0.0, 0.01, 0.03, 0.05]
visibility = [10.0, 8.0, 5.0, 4.0, 3.0, 2.0 ]
visibility_output = [ [ [] for n in visibility ] for n in noise_values_visibility ]
for repetition in range(50):
    start = time.time()
    for n in range(len(noise_values_visibility)):
        for v in range(len(visibility)):
            sim = Simulator(4)  # 4 robots
            steps, collisions = sim.run(mode="aware", current_noise=noise_values_visibility[n], detection_range=visibility[v], identifier="oracle")
            visibility_output[n][v].append((steps,collisions))
    end = time.time()
    print("Step %d done, took %f" % (repetition, (end-start)) )

In [ ]:
# Plot of min, median and max time taken to get to the goal, 
# divided up by noise in the system and mode of operation of the robots
fig, axes = plt.subplots(2,2)
outsideX = ["Collisions", "Time"]
outsideY = ["Noise: %.2f" % n for n in [0.0,0.03] ]
pad = 5 # in points
for ax, col in zip(axes[0], outsideX):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')
for ax, row in zip(axes[:,0], outsideY):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center', rotation='vertical')
plt.setp(axes[:,0], xlabel='Visbility range', ylabel='Collisions')
plt.setp(axes[:,1], xlabel='Visbility range', ylabel='Time') 
for n in range(len(noise_values_visibility)):
    if n % 2 != 1:
        continue
    data = visibility_output[n]
    mine = []
    med = []
    maxe = []
    # Time plots
    for m in range(len(visibility)):
        val = data[m]
        mine.append(np.amin(val, axis=0)[0] / 10.0)
        med.append(np.median(val, axis=0)[0]/ 10.0)
        maxe.append(np.amax(val, axis=0)[0]/ 10.0)
    axes[n/2][1].set_xticks(visibility)
    axes[n/2][1].plot(visibility, mine, 'rv', visibility, med, 'bo', visibility, maxe, 'g^')
    mine = []
    med = []
    maxe = []
    # Collision plots
    for m in range(len(visibility)):
        val = data[m]
        mine.append(np.amin(val, axis=0)[1])
        med.append(np.median(val, axis=0)[1])
        maxe.append(np.amax(val, axis=0)[1])
    axes[n/2][0].set_xticks(visibility)
    axes[n/2][0].plot(visibility, mine, 'rv', visibility, med, 'bo', visibility, maxe, 'g^')


fig.tight_layout()
# tight_layout doesn't take these labels into account. We'll need 
# to make some room. These numbers are are manually tweaked. 
# You could automatically calculate them, but it's a pain.
fig.subplots_adjust(left=0.25, top=0.95)
fig.savefig('collisions_and_time_over_visibility_and_noise.eps', bbox_inches='tight',  format='eps', dpi=300)
plt.show()